## Generation of the GT for Training

In [1]:
import math
import sys
import time
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import spatial

from tqdm.auto import tqdm, trange
from tqdm.contrib.concurrent import process_map, thread_map

from sphere import Sphere
from visualize import Visualize
from semantic_classes import SemanticClasses
    
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# NUSCENES LOADER
# ----------------------------------------------------------------

# export_ds = '/media/berlukas/Data2/datasets/nuscenes/processed/'
export_ds = '/media/scratch/berlukas/nuscenes'
sem_clouds_filename = f"{export_ds}/clouds1_bw120_range_sem.npy"

print(f"Loading from sem clouds from {sem_clouds_filename}")
sem_cloud_features = np.load(sem_clouds_filename)
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading from sem clouds from /media/scratch/berlukas/nuscenes/clouds1_bw120_range_sem.npy
Shape of sem clouds is (11230, 2, 240, 240)


In [2]:
# KITTI LOADER
# ----------------------------------------------------------------

export_ds = '/media/scratch/berlukas/nuscenes/kitti_processed'
sem_clouds_filename = f"{export_ds}/clouds-08.npy"

print(f"Loading from sem clouds from {sem_clouds_filename}")
sem_cloud_features = np.load(sem_clouds_filename)
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading from sem clouds from /media/scratch/berlukas/nuscenes/kitti_processed/clouds-08.npy
Shape of sem clouds is (4071, 3, 200, 200)


In [15]:
# WAYMO LOADER
# ----------------------------------------------------------------

export_ds = '/media/scratch/berlukas/waymo/extracted/features_120'
archives = os.listdir(export_ds)
sem_cloud_features = None
for archive in archives:
    sem_clouds_filename = f"{export_ds}/{archive}"
    
    print(f"Loading from sem clouds from {sem_clouds_filename}")
    if sem_cloud_features is None:
        sem_cloud_features = np.load(sem_clouds_filename)
    else:
        features = np.load(sem_clouds_filename)
        sem_cloud_features = np.concatenate((sem_cloud_features, features))
    
# Remove intensity channel
sem_cloud_features = sem_cloud_features[:, [0, 2], :, :]
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading from sem clouds from /media/scratch/berlukas/waymo/extracted/features_120/features01.npy
Loading from sem clouds from /media/scratch/berlukas/waymo/extracted/features_120/features02.npy
Loading from sem clouds from /media/scratch/berlukas/waymo/extracted/features_120/features00.npy
Shape of sem clouds is (2237, 2, 240, 240)


In [ ]:
# only processing val
# n_clouds = sem_cloud_features.shape[0]
# print(f'we have in total {n_clouds}')
# n_train = n_clouds - 6053
# n_val = n_clouds - n_train
# print(f'we will keep {n_train} for training and {n_val} for validation.')
# sem_cloud_features = sem_cloud_features[n_train:, :, :, :]

In [3]:
n_clouds = sem_cloud_features.shape[0]
idx_sem = 1
classes = [None] * n_clouds
for i in tqdm(range(0, n_clouds)):
    sem_cloud = np.reshape(sem_cloud_features[i,:,:,:], (2, -1)).T    
    classes[i] = sem_cloud[:, idx_sem]
classes = np.array(classes[:])
print(f"shape of classes is {classes.shape}")
unique_classes = np.unique(classes)
n_unique_classes = len(unique_classes)
print(f"we have {n_unique_classes} classes in the dataset. That is:")
print(unique_classes)


shape of classes is (11230, 57600)
we have 33 classes in the dataset. That is:
[-1.  0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.
 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31.]


In [ ]:
n_clouds = sem_cloud_features.shape[0]
idx_sem = 2
classes = [None] * n_clouds
for i in tqdm(range(0, n_clouds)):
    sem_cloud = np.reshape(sem_cloud_features[i,:,:,:], (3, -1)).T    
    classes[i] = sem_cloud[:,idx_sem]
classes = np.array(classes[:])
print(f"shape of classes is {classes.shape}")
unique_classes = np.unique(classes)
n_unique_classes = len(unique_classes)
print(f"we have {n_unique_classes} classes in the dataset.")
print(unique_classes)

## Iterate over the features and generate the GT

In [5]:
n_features = sem_cloud_features.shape[0]
n_rows = sem_cloud_features.shape[2]
n_cols = sem_cloud_features.shape[3]
idx_sem = 1

print(f'Iterating for {n_features} features and data of size: ({n_rows}/{n_cols})')
for feature in tqdm(range(n_features)):
    for i in range(n_rows):
        for j in range(n_cols):
            cur_class = int(sem_cloud_features[feature, idx_sem, i, j])
#             mapped_class = SemanticClasses.map_nuscenes_label_old(cur_class)
            mapped_class = SemanticClasses.map_nuscenes_label(cur_class)
#             mapped_class = SemanticClasses.map_waymo_to_nuscenes_label(cur_class)
            sem_cloud_features[feature, idx_sem, i, j] = mapped_class

Iterating for 11230 features and data of size: (240/240)


In [5]:
# sem_cloud_features = sem_cloud_features[0:10,:,:,:]
idx_sem = 2
n_data = sem_cloud_features.shape[0]
n_features = sem_cloud_features.shape[1]
n_rows = sem_cloud_features.shape[2]
n_cols = sem_cloud_features.shape[3]

idx_intensity = 1;
for feature in tqdm(range(n_data)):
    for i in range(n_rows):
        for j in range(n_cols):
            cur_class = int(sem_cloud_features[feature, idx_sem, i, j])            
            mapped_class = SemanticClasses.map_kitti_to_nuscenes_label(cur_class)
            
            cur_intensity = sem_cloud_features[feature, idx_intensity, i, j]
#             if cur_intensity > 0:
#                 cur_intensity = ((cur_intensity ) / 0.16)
#                 cur_intensity = cur_intensity * 255 
            
#             mapped_class = map_nuscenes_label(cur_class)
#             mapped_class = map_nuscenes_label_old(cur_class)
            sem_cloud_features[feature, idx_sem, i, j] = mapped_class
#             sem_cloud_features[feature, idx_intensity, i, j] = int(cur_intensity)
#             if cur_intensity > 1:
#                 print(sem_cloud_features[feature, idx_intensity, i, j])

In [7]:
filename = 'sem_clouds1_nuscenes_bw120'
sem_clouds_gt_filename = f'{export_ds}/{filename}.npy'
sem_clouds_gt_tiny_filename = f'{export_ds}/{filename}_tiny.npy'
np.save(sem_clouds_gt_filename, sem_cloud_features)

n_process = 400
np.save(sem_clouds_gt_tiny_filename, sem_cloud_features[0:n_process, :, :, :])

print(f"Wrote features to {sem_clouds_gt_filename}")
print(f"Wrote tiny features to {sem_clouds_gt_tiny_filename}")

Wrote features to /media/scratch/berlukas/nuscenes/sem_clouds1_nuscenes_bw120.npy
Wrote tiny features to /media/scratch/berlukas/nuscenes/sem_clouds1_nuscenes_bw120_tiny.npy


In [6]:
# Remove a few samples for validation
n_clouds = sem_cloud_features.shape[0]
print(f'we have in total {n_clouds}')
n_train = n_clouds - 400
n_val = n_clouds - n_train
print(f'we will keep {n_train} for training and {n_val} for validation.')

print(f'init shape is {sem_cloud_features.shape}')
sem_train = sem_cloud_features[:n_train, :, :, :]
sem_val = sem_cloud_features[n_train:, :, :, :]
print(f'training shape is {sem_train.shape}, validation shape is {sem_val.shape}')

np.save(f'{export_ds}/sem_clouds3.npy', sem_train)
np.save(f'{export_ds}/sem_clouds_val_200.npy', sem_val)

we have in total 11230
we will keep 10830 for training and 400 for validation.
init shape is (11230, 2, 240, 240)
training shape is (10830, 2, 240, 240), validation shape is (400, 2, 240, 240)


In [9]:
sem_val_tiny = sem_val[:500, :, :, :]
np.save(f'{export_ds}/sem_clouds_val_tiny.npy', sem_val_tiny)